In [1]:
from bs4 import BeautifulSoup


In [2]:
import requests
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [3]:
source= requests.get(url).text


In [4]:
soup= BeautifulSoup(source,'lxml')


In [5]:
table=soup.find('table')


In [6]:
import pandas as pd


In [7]:
tables = pd.read_html(source)


In [8]:
df = pd.DataFrame(tables[0])
df.columns=['Postcode','Borough','Neighbourhood']
df
df1=df.drop(0)


In [9]:
df1.head()


,Postcode,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [10]:
df1=df1[df1.Borough != 'Not assigned']

df1.head(10)
df_new=pd.DataFrame(df1)
df_new.head()
df2=df1.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2=df2.reset_index()
df2.head(20)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview"
7,M1L,Scarborough,"Clairlea,Oakridge,Golden Mile"
8,M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [11]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

len(df2[df2['Neighbourhood'] == 'Not assigned'])


0

In [12]:
df2.shape


(103, 3)

In [13]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
y = df_data_1[df_data_1.columns[-2:]]


In [47]:
y.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [48]:


# place the DataFrames side by side
df = pd.concat([df2, y], axis=1)

In [49]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [56]:
!conda install -c conda-forge folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py37_0         148 KB  conda-forge
    folium-0.9.1               |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         207 KB

The following packages will be UPDATED:

    ca-certificates: 2019.5.15-0                   --> 2019.6.16-hecc5488_0 conda-forge
    certifi:         2019.6.16-py37_0              --> 2019.6.16-py37_0     conda-forge
    folium:          0.5.0-py_0        conda-forge --> 0.9.1-py_0           conda-forge

The following packages will be DOWNGRADED:

    openssl:         1.1.1c-h7b6447c_1             --> 1.1.1b-h14c3975_1    conda

In [ ]:

import folium
# Find the center of all the locations and prepare the folium map

center_lat=df.Latitude.mean()
center_long=df.Longitude.mean()

venues_map = folium.Map(location=[center_lat, center_long], zoom_start=13) # generate map centred around the Conrad Hotel

df2=df1.head(80)
for lat, lng, label in zip(df.Latitude, df.Longitude, df.Borough):
    #print(lat,lng,label)
    folium.features.CircleMarker(
            [lat, lng],
            radius=2,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
    ).add_to(venues_map)
    